## RAG 평가
- 크게는 2가지 평가 방법론
  - (1) LLM + RAG의 end-to-end 성능 평가
    - 이전 파트에서 여러 번 진행 (ex. LLM-as-a-judge...)
  - (2) RAG 로직의 성능 평가
    - 2번에 집중 예정

In [2]:
import pickle

with open('./res/rag_data.pkl', 'rb') as f:
    rag_data = pickle.load(f)

In [3]:
rag_data['questions'][0]

'글로벌 저금리 현상이 부각된 원인은 무엇인가요?'

In [5]:
rag_data['contexts'][0]

['... 변화의 동인은 크게 두 가지로 분류할\n수 있다. 하나는 금융업계 전반에 부각되는 디지털화(digitalization)1)이고\n다른 하나는 저금리·저성장으로 대표되는 뉴노멀(new normal)이다.  ... . 뉴노멀의 특징으로는 금융투자\n및 자산운용의 어려움이 가중되는 경향, 고수익-고위험 부문으로의 쏠림\n현상 등의 위험요인과 함께 자산관리서비스 수요의 증가 등 대응 노력을\n살펴본다. 제III장은 디지털화 및 뉴노멀에 대응하는 해외 사례들을 살펴본다.  ... 뉴노멀의\n주요 특징인 저성장, 저금리 등은 일본의 금융투자업계가 30년 전부터 고민해\n온 문제이기 때문이다.  ... 제III장에서 살펴본 해외\n사례들에서 살펴본 금융투자업계 변화의 규칙성이 어느 정도 유지될 것이라는\n1 디지털화(digitalization)에 대한 ...',
 '3. 문제점\n1) 기업신용위험 평가항목의 적정성\n글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로\n금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을\n측정하는 데 적정한 지표인지에 대한 검토가 필요\n기업신용위험 정 ... 작용할 것으로 예상\n저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배\n율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이\n발생\n더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기\n연장 및 이자상환 유예조치로 이 ... 는 추세\n이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이\n있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...',
 '2. 뉴노멀의 도래(신 3저의 도래)\n최근 코로나 사태를 포함하여 2008년 글로벌 금융위기 이후 전세계적인\n저금리·저성장 상황\n∙ 글로벌 금융위기를 극복하는 과정에서 미국, 유럽, 일본 등 주요국의\n완화적 통화정책으로 글로벌 저금리 현상이 부각\n∙ 코로나 팬데믹이 진정되더라도 글로벌 저금리 추세가 지속되는 가운\n데 우리나라의 저금리 기조도 장기화될 가능

### 정답 Context가 있는 경우
- Accuracy
  - 정답이 있는 상황이라 가장 정확한 평가
    - Reference 또는 정답이 있을 때만 사용 가능
    - 실제로는 없는 경우가 대부분

In [6]:
from utils import get_embedding, cosine_similarity


embed_q = get_embedding(rag_data['questions'][0])
embed_c0 = get_embedding(rag_data['contexts'][0][0])
embed_c1 = get_embedding(rag_data['contexts'][0][1])
embed_c2 = get_embedding(rag_data['contexts'][0][2])

In [14]:
print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))

0.3622209587156101
0.4410565664426629
0.4155904417828041


In [13]:
rag_data['contexts_answer_idx'][0]

2

In [15]:
embed_q = get_embedding(rag_data['questions'][0], model='text-embedding-3-large')
embed_c0 = get_embedding(rag_data['contexts'][0][0], model='text-embedding-3-large')
embed_c1 = get_embedding(rag_data['contexts'][0][1], model='text-embedding-3-large')
embed_c2 = get_embedding(rag_data['contexts'][0][2], model='text-embedding-3-large')

print(cosine_similarity(embed_q, embed_c0))
print(cosine_similarity(embed_q, embed_c1))
print(cosine_similarity(embed_q, embed_c2))

0.4111640101857708
0.39904782383917453
0.5323605266350249


In [18]:
from tqdm import tqdm

num_questions = 10
num_contexts = 3

top_context_indices = []

for i in tqdm(range(num_questions)):
    embed_q = get_embedding(rag_data['questions'][i])

    similarities = []
    for j in range(num_contexts):
        embed_c = get_embedding(rag_data['contexts'][i][j])
        similarity = cosine_similarity(embed_q, embed_c)
        similarities.append(similarity)

    top_context_index = similarities.index(max(similarities))
    top_context_indices.append(top_context_index)

print(f"Top Context Indices: {top_context_indices}")

100%|██████████| 10/10 [00:18<00:00,  1.83s/it]

Top Context Indices: [1, 1, 0, 1, 1, 1, 2, 0, 1, 1]


In [20]:
rag_data['contexts_answer_idx'][:num_questions]

[2, 1, 0, 2, 2, 1, 2, 2, 1, 1]

In [19]:
def calculate_accuracy(predicted, actual):
    correct = sum(p == a for p, a in zip(predicted, actual))
    total = len(predicted)
    
    accuracy = correct / total
    return accuracy

accuracy = calculate_accuracy(top_context_indices, rag_data['contexts_answer_idx'][:10])
print(f"Accuracy: {accuracy:.2%}")

Accuracy: 60.00%


### 정답 Context가 없는 경우
- RAGAS
  - GPT-4를 활용한 RAG 로직 평가용 라이브러리
    - RAG 뿐만 아니라 LLM 단독 그리고 LLM + RAG 평가도 가능
  - RAG 평가의 경우 정답 Context가 있어도 사용이 가능하지만 정답이 없는 경우에도 사용이 가능
  - OpenAI에서도 RAG 로직 평가를 위해 사용 (OpenAI Dev Day)
  - 기본으로 제공하는 평가용 System Prompt가 영문 기반이라 한글 평가는 상대적으로 정확도가 약간 떨어지는 편

In [36]:
contexts_predictions = []
for i in range(len(top_context_indices)):
    index = top_context_indices[i]
    contexts_predictions.append([rag_data['contexts'][i][index]])
contexts_predictions

[['3. 문제점\n1) 기업신용위험 평가항목의 적정성\n글로벌 금융위기 이후 저금리 기조가 지속되었고, 최근 추가적으로\n금리가 하락함에 따라 이자보상배율 등의 지표들이 기업의 부실을\n측정하는 데 적정한 지표인지에 대한 검토가 필요\n기업신용위험 정 ... 작용할 것으로 예상\n저금리 기조가 지속되는 가운데 금융비용이 감소하여 이자보상배\n율이 기업의 부실을 가늠하는 지표로 적합한지에 대해 의문이\n발생\n더불어 코로나19 사태로 인한 중소기업에 대한 금융권 대출 만기\n연장 및 이자상환 유예조치로 이 ... 는 추세\n이러한 졸업비율 감소 현상의 원인으로 저조한 경기의 영향이\n있었던 것으로 추정되지만 개시연도를 기준으로 졸 ...'],
 ['축소되어 자산운용의 어려움이 가중\n∙ 투자자 입장에서는 원하는 수익률을 얻기 어려워 보다 높은 수익률을\n추구하는 투자행태 ... 나. 고수익-고위험 부문으로 쏠림현상\n금융회사, 가계 등의 수익률 추구 성향(search for yield)이 강화되어\n위험자산으로 과도한 자금 유입이 발생할 수 있음.\n∙ 저금리 기조 하에서는 투자자의 고수익 추구로 인해 고위험 파생상\n품, 부동산금융 등 리스크가 큰 분야로의 쏠림현상 심화 가능성\n∙ 사모펀드 부문의 빠른 성장, 파생상품 자산의 급증 등 개인의 투자위\n험도 증가하는 추세\n* 고수익-고위험 추구  ...  등)으로 기대수익률을 충족시키지\n못해 고위험-고수익을 추구할 가능성 증가\n∙ 개인 투자자들은 이에 대한 전문성 부족으로  ...'],
 ['1. 디지털화의 진전과 금융투자업의 변화 : 20세기 이후\n가. 1950~60년대\nMorrison and Wilhelm(2007), Jensen(1993) 등은 컴퓨터 기술의\n발전과 디지털화의 진전이 금융투자업의 사업모형과 산업구조에 큰\n영향을 끼치는 것은 20세기 중반 이후로 평가\n1960년대 후반 소위 백오피스 위기를 거치면서 백오피스 업무, 거래소\n거래 중심의 전산화가 가속 ... 1980년대 이후 

- Context Recall
  - RAG 모델이 가져온 Context 문장에서 질문의 정답 문장이 얼마나 많이 존재하는 지 확인
  - 실제 질문의 답변과 Context 정보만 사용 (정답 Context X)

In [37]:
from datasets import Dataset 
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_relevancy, context_recall, context_precision


data_samples = {
    'question': rag_data['questions'][:num_questions],
    'contexts' : contexts_predictions,
    'ground_truth': rag_data['answers'][:num_questions]
}

dataset = Dataset.from_dict(data_samples)

score = evaluate(dataset, metrics=[context_recall])
score

Evaluating: 100%|██████████| 10/10 [00:17<00:00,  1.71s/it]


{'context_recall': 0.5467}